In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API key
from api_keys import d_key

### Extract CSVs into DataFrames

In [2]:
# Creates lists for data to be stored 
id = []
name = []
weight = []
height = []
bred_for = []
breed_group = []

#importing API 
url = r"https://api.thedogapi.com/v1/breeds"
response = requests.get(url).json()

pprint(response[0])

{'bred_for': 'Small rodent hunting, lapdog',
 'breed_group': 'Toy',
 'height': {'imperial': '9 - 11.5', 'metric': '23 - 29'},
 'id': 1,
 'image': {'height': 1199,
           'id': 'BJa4kxc4X',
           'url': 'https://cdn2.thedogapi.com/images/BJa4kxc4X.jpg',
           'width': 1600},
 'life_span': '10 - 12 years',
 'name': 'Affenpinscher',
 'origin': 'Germany, France',
 'reference_image_id': 'BJa4kxc4X',
 'temperament': 'Stubborn, Curious, Playful, Adventurous, Active, Fun-loving',
 'weight': {'imperial': '6 - 13', 'metric': '3 - 6'}}


In [3]:
# looping through the API to display specific data 
x=0
while (x) < 4:
    #pprint(response[x])
    print(f"id: {response[x]['id']}")
    print(f"Weight (Kg): {response[x]['weight']['metric']}")
    print(f"Height (cm): {response[x]['height']['metric']}")
    if 'bred_for' in response[x]:
        if not response[x]['bred_for']:
            print("Bred for: Unknown")
        else:
            print(f"Bred for: {response[x]['bred_for']}")
    else:
        print("Bred for: Unknown")
    if 'breed_group' in response[x]:
        if not response[x]['breed_group'] :
            print('Breed Group: Unknown')
        else:
            print(f"Breed Group: {response[x]['breed_group']}")
    else:
        print('Breed Group: Unknown')
    print('....')
    x += 1

id: 1
Weight (Kg): 3 - 6
Height (cm): 23 - 29
Bred for: Small rodent hunting, lapdog
Breed Group: Toy
....
id: 2
Weight (Kg): 23 - 27
Height (cm): 64 - 69
Bred for: Coursing and hunting
Breed Group: Hound
....
id: 3
Weight (Kg): 20 - 30
Height (cm): 76
Bred for: A wild pack animal
Breed Group: Unknown
....
id: 4
Weight (Kg): 18 - 29
Height (cm): 53 - 58
Bred for: Badger, otter hunting
Breed Group: Terrier
....


In [4]:
#appending the data to the empty lists 
for i in response:
    id.append(i["id"])
    name.append(i["name"])
    weight.append(i["weight"]['metric'])
    height.append(i["height"]['metric'])
    if 'breed_group' in i:
        if not i['breed_group']:
            breed_group.append('Unknown')
        else:
            breed_group.append(i['breed_group'])
    else:
        breed_group.append('Unknown')
    if 'bred_for' in i:
        if not i['bred_for']:
            bred_for.append('Unknown')
        else:
            bred_for.append(i['bred_for'])
    else:
        bred_for.append('Unknown')    
        
#test to ensure lengths are the same
print(len(bred_for))
print(len(breed_group))

# create a dictionary for establishing dataframe
dog_data = {
    "ID":id,
    "breed":name,
    "weight": weight,
    "height": height,
    "breed_group": breed_group,
    "bred_for":bred_for,
}

dog_df = pd.DataFrame(dog_data)

dog_df.head()

172
172


,ID,breed,weight,height,breed_group,bred_for
0,1,Affenpinscher,3 - 6,23 - 29,Toy,"Small rodent hunting, lapdog"
1,2,Afghan Hound,23 - 27,64 - 69,Hound,Coursing and hunting
2,3,African Hunting Dog,20 - 30,76,Unknown,A wild pack animal
3,4,Airedale Terrier,18 - 29,53 - 58,Terrier,"Badger, otter hunting"
4,5,Akbash Dog,41 - 54,71 - 86,Working,Sheep guarding


In [5]:
#importing the dog intelligence CSV 
intelligence = "Resources/intelligence.csv"
intelligence_df = pd.read_csv(intelligence)
intelligence_df.head()


,Breed,Classification,obey,reps_lower,reps_upper
0,Border Collie,Brightest Dogs,95%,1,4
1,Poodle,Brightest Dogs,95%,1,4
2,German Shepherd,Brightest Dogs,95%,1,4
3,Golden Retriever,Brightest Dogs,95%,1,4
4,Doberman Pinscher,Brightest Dogs,95%,1,4


In [6]:

intelligence_df.rename(columns={"Breed": "breed", "Classification": "classification"},inplace = True)
intelligence_df.head()

,breed,classification,obey,reps_lower,reps_upper
0,Border Collie,Brightest Dogs,95%,1,4
1,Poodle,Brightest Dogs,95%,1,4
2,German Shepherd,Brightest Dogs,95%,1,4
3,Golden Retriever,Brightest Dogs,95%,1,4
4,Doberman Pinscher,Brightest Dogs,95%,1,4


In [7]:
#Transforming the data to Postgres 
rds_connection_string = "postgres:postgres@localhost:5432/breeds_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_6768/680784670.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['intelligence_info', 'breed_info']

In [8]:
#connect intelligence table to postgres
intelligence_df.to_sql(name='intelligence_info', con=engine, if_exists='append', index=False)
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_6768/950548486.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['intelligence_info', 'breed_info']

In [9]:
#testing the intelligence table after connecting to ensure connection worked
pd.read_sql_query('select * from intelligence_info', con=engine).head()

,breed,classification,obey,reps_lower,reps_upper
0,Border Collie,Brightest Dogs,95%,1,4
1,Poodle,Brightest Dogs,95%,1,4
2,German Shepherd,Brightest Dogs,95%,1,4
3,Golden Retriever,Brightest Dogs,95%,1,4
4,Doberman Pinscher,Brightest Dogs,95%,1,4


In [10]:
#connect breeds table to postgres
dog_df.to_sql(name='breed_info', con=engine, if_exists='append', index=False)
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_6768/2110304788.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['intelligence_info', 'breed_info']

In [11]:
#testing the breeds table after connecting to ensure connection worked
pd.read_sql_query('select * from breed_info', con=engine).head()

,ID,breed,weight,height,breed_group,bred_for
0,1,Affenpinscher,3 - 6,23 - 29,Toy,"Small rodent hunting, lapdog"
1,2,Afghan Hound,23 - 27,64 - 69,Hound,Coursing and hunting
2,3,African Hunting Dog,20 - 30,76,Unknown,A wild pack animal
3,4,Airedale Terrier,18 - 29,53 - 58,Terrier,"Badger, otter hunting"
4,5,Akbash Dog,41 - 54,71 - 86,Working,Sheep guarding
